#IIC-2433 Minería de Datos UC

* Versiones de librerías, python 3.8.10
* pandas 1.5.3
* tensorflow/keras 2.12.0

In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical

# 1. Carga de la base de datos

En esta tarea se trabajará con un dataset de vinos obtenido de Kaggle:
Cárguelo, léalo y muéstrelo.

https://www.kaggle.com/datasets/yasserh/wine-quality-dataset

In [2]:
dframe = pd.read_csv("WineQT.csv", encoding = "ISO-8859-1")
dframe.shape

(1143, 13)

In [3]:
dframe

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Id
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,0
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,1
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,2
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,3
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6,1592
1139,6.8,0.620,0.08,1.9,0.068,28.0,38.0,0.99651,3.42,0.82,9.5,6,1593
1140,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5,1594
1141,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6,1595


# 2. Preprocesamiento

Realice el preprocesamiento que considere adecuado para este *dataset* y argumente todas sus decisiones.

In [4]:
 #Chequeamos si hay algun valor nulo en el df, y si hay eliminamos la fila
if dframe.isnull().values.any():
    dframe.dropna()

In [5]:
# Calcular el rango intercuartílico
Q1 = dframe.quantile(0.25)
Q3 = dframe.quantile(0.75)
IQR = Q3 - Q1

# Definir los límites inferior y superior para detectar outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filtrar los outliers
filtered_df = dframe[~((dframe < lower_bound) | (dframe > upper_bound)).any(axis=1)]
filtered_df


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Id
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,0
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,1
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,2
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,3
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6,1592
1139,6.8,0.620,0.08,1.9,0.068,28.0,38.0,0.99651,3.42,0.82,9.5,6,1593
1140,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5,1594
1141,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6,1595


In [6]:
# Prediceremos Quality
X = filtered_df.drop('quality', axis=1)
y = filtered_df['quality']
y = to_categorical(y)

# 3. Preguntas iniciales


Deberá encontrar la mejor combinación de hiperparámetros que le permita obtener el modelo con mejor *accuracy*.

A priori, ¿cuáles cree que serán las mejores combinaciones de hiperparámetros para este problema en particular? ¿Por qué?

- Un patience alto puede ayudar a obtener un mejor modelo, dado que define cuantos epochs consecutivos pueden haber sin mejora, pero si despues logra "desestancarse" seguira mejorando 
- un alto numero de epochs combinado con el parametro que dije antes.
- El numero de capas neuronales para aumentar la complejidad del modelo (aunque a su vez afecta su velocidad y uso de recursos)

# 3. Modelos

Ocupando la librería **keras** de tensorflow, construya redes neuronales multicapas variando los siguientes hiperparámetros: función de activación, cantidad de neuronas por cada capa, cantidad de capas, optimizador y cantidad de épocas.

### INTENTO 1

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = Sequential()
model.add(Dense(64, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

callbacks = [EarlyStopping(monitor='val_loss', patience=3)]

model.fit(X_train, y_train, validation_split=0.2, epochs=300, callbacks=callbacks)

y_train_pred = np.argmax(model.predict(X_train), axis=-1)
y_test_pred = np.argmax(model.predict(X_test), axis=-1)

y_train_label = np.argmax(y_train, axis=-1)
y_test_label = np.argmax(y_test, axis=-1)

Epoch 1/300
17/17 [==============================] - 1s 8ms/step - loss: 2.0105 - accuracy: 0.1839 - val_loss: 1.8247 - val_accuracy: 0.4254
Epoch 2/300
17/17 [==============================] - 0s 2ms/step - loss: 1.6849 - accuracy: 0.4747 - val_loss: 1.4342 - val_accuracy: 0.4701
Epoch 3/300
17/17 [==============================] - 0s 3ms/step - loss: 1.2890 - accuracy: 0.5159 - val_loss: 1.0897 - val_accuracy: 0.5522
Epoch 4/300
17/17 [==============================] - 0s 3ms/step - loss: 1.0531 - accuracy: 0.5797 - val_loss: 0.9379 - val_accuracy: 0.5672
Epoch 5/300
17/17 [==============================] - 0s 3ms/step - loss: 0.9493 - accuracy: 0.6079 - val_loss: 0.8813 - val_accuracy: 0.5821
Epoch 6/300
17/17 [==============================] - 0s 3ms/step - loss: 0.8967 - accuracy: 0.6229 - val_loss: 0.8630 - val_accuracy: 0.6119
Epoch 7/300
17/17 [==============================] - 0s 3ms/step - loss: 0.8622 - accuracy: 0.6323 - val_loss: 0.8585 - val_accuracy: 0.6194
Epoch 8/300
1

In [20]:
print("Train Accuracy: ", accuracy_score(y_train_label, y_train_pred))

print("\nTrain Classification Report:")
print(classification_report(y_train_label, y_train_pred,zero_division=0))

Train Accuracy:  0.6701649175412294

Train Classification Report:
              precision    recall  f1-score   support

           4       0.00      0.00      0.00        17
           5       0.67      0.82      0.74       297
           6       0.67      0.59      0.62       280
           7       0.66      0.55      0.60        73

    accuracy                           0.67       667
   macro avg       0.50      0.49      0.49       667
weighted avg       0.65      0.67      0.66       667



Lo logre altiro xd

### INTENTO 2

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = Sequential()
model.add(Dense(128, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

callbacks = [EarlyStopping(monitor='val_loss', patience=5)]

model.fit(X_train, y_train, validation_split=0.2, epochs=20, callbacks=callbacks)

y_train_pred = np.argmax(model.predict(X_train), axis=-1)
y_test_pred = np.argmax(model.predict(X_test), axis=-1)

y_train_label = np.argmax(y_train, axis=-1)
y_test_label = np.argmax(y_test, axis=-1)

Epoch 1/20
15/15 [==============================] - 1s 10ms/step - loss: 2.0822 - accuracy: 0.0730 - val_loss: 1.9496 - val_accuracy: 0.2991
Epoch 2/20
15/15 [==============================] - 0s 3ms/step - loss: 1.8630 - accuracy: 0.3734 - val_loss: 1.6951 - val_accuracy: 0.5470
Epoch 3/20
15/15 [==============================] - 0s 3ms/step - loss: 1.5957 - accuracy: 0.5107 - val_loss: 1.3621 - val_accuracy: 0.5897
Epoch 4/20
15/15 [==============================] - 0s 2ms/step - loss: 1.3202 - accuracy: 0.5773 - val_loss: 1.1717 - val_accuracy: 0.5641
Epoch 5/20
15/15 [==============================] - 0s 3ms/step - loss: 1.1738 - accuracy: 0.5901 - val_loss: 1.0953 - val_accuracy: 0.5897
Epoch 6/20
15/15 [==============================] - 0s 3ms/step - loss: 1.0701 - accuracy: 0.6116 - val_loss: 1.0448 - val_accuracy: 0.6239
Epoch 7/20
15/15 [==============================] - 0s 3ms/step - loss: 0.9588 - accuracy: 0.6373 - val_loss: 0.9876 - val_accuracy: 0.6154
Epoch 8/20
15/15 [=

In [24]:
print("Train Accuracy: ", accuracy_score(y_train_label, y_train_pred))

print("\nTrain Classification Report:")
print(classification_report(y_train_label, y_train_pred,zero_division=0))

Train Accuracy:  0.7204116638078902

Train Classification Report:
              precision    recall  f1-score   support

           4       0.00      0.00      0.00        14
           5       0.72      0.83      0.77       261
           6       0.70      0.66      0.68       241
           7       0.81      0.64      0.72        67

    accuracy                           0.72       583
   macro avg       0.56      0.53      0.54       583
weighted avg       0.71      0.72      0.71       583



### INTENTO 3

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = Sequential()
model.add(Dense(64, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

callbacks = [EarlyStopping(monitor='val_loss', patience=10)]

model.fit(X_train, y_train, validation_split=0.2, epochs=40, callbacks=callbacks)

y_train_pred = np.argmax(model.predict(X_train), axis=-1)
y_test_pred = np.argmax(model.predict(X_test), axis=-1)

y_train_label = np.argmax(y_train, axis=-1)
y_test_label = np.argmax(y_test, axis=-1)

Epoch 1/40
15/15 [==============================] - 1s 10ms/step - loss: 1.9554 - accuracy: 0.3026 - val_loss: 1.8135 - val_accuracy: 0.4188
Epoch 2/40
15/15 [==============================] - 0s 3ms/step - loss: 1.7275 - accuracy: 0.4571 - val_loss: 1.6137 - val_accuracy: 0.4188
Epoch 3/40
15/15 [==============================] - 0s 3ms/step - loss: 1.5008 - accuracy: 0.4592 - val_loss: 1.3765 - val_accuracy: 0.4615
Epoch 4/40
15/15 [==============================] - 0s 3ms/step - loss: 1.2748 - accuracy: 0.5236 - val_loss: 1.1411 - val_accuracy: 0.5556
Epoch 5/40
15/15 [==============================] - 0s 2ms/step - loss: 1.1023 - accuracy: 0.5794 - val_loss: 1.0280 - val_accuracy: 0.5641
Epoch 6/40
15/15 [==============================] - 0s 3ms/step - loss: 1.0276 - accuracy: 0.6180 - val_loss: 0.9839 - val_accuracy: 0.5812
Epoch 7/40
15/15 [==============================] - 0s 3ms/step - loss: 0.9820 - accuracy: 0.6245 - val_loss: 0.9710 - val_accuracy: 0.5641
Epoch 8/40
15/15 [=

In [26]:
print("Train Accuracy: ", accuracy_score(y_train_label, y_train_pred))

print("\nTrain Classification Report:")
print(classification_report(y_train_label, y_train_pred,zero_division=0))

Train Accuracy:  0.7135506003430532

Train Classification Report:
              precision    recall  f1-score   support

           4       0.00      0.00      0.00        14
           5       0.71      0.84      0.77       261
           6       0.72      0.64      0.68       241
           7       0.70      0.63      0.66        67

    accuracy                           0.71       583
   macro avg       0.53      0.53      0.53       583
weighted avg       0.70      0.71      0.70       583



Intente encontrar un modelo que obtenga un 66.6% de *accuracy*. ¿Lo logró? ¿por qué?

Se logro, 

Una vez obtenido el mejor clasificador, ¿cuáles son sus errores más comunes? ¿cómo se podrían mejorar?

# 4. Preguntas finales

¿Qué resultados obtuvo? Se habrá dado cuenta que encontrar el mejor modelo no es lo mismo que buscar los mejores hiperparámetros unilateralmente. ¿Cuáles combinaciones funcionaron mejor y cuáles peor? ¿Por qué cree que fue así? Argumente.